In [1]:
from tramway.helper.tessellation import *
from tramway.helper.inference import *
from tramway.plot.mesh import plot_delaunay
import matplotlib.pyplot as plt
from scipy.stats import skellam
from scipy.optimize import fmin,minimize

## script main parameters ##

positions_file = '/Users/cvesterg/ownCloud/Random walks/Mapping without tracking/BP_data/Rac1/15_06_2016/reference/c01.txt'
positions_file = '/Users/cvesterg/ownCloud/Random walks/Mapping without tracking/BP_data/MWT_Rac1/4/trajectories.txt'
trajectories_bool=True


# average number of locations per cell
#location_count = 10
# minimum number of locations per cell
#min_location_count = 2
# start and end times to consider for fit
#t_start=1.00; t_end=1.04

# in the same unit as in the trajectory file (here in micrometers)
ref_distance=0.1
localization_error = 0.03
dt=0.04
#jeffreys_prior = False
#diffusivity_smoothing
# keep it low otherwise the inference will take days
max_iter = 100


############################

rwa_file = os.path.splitext(positions_file)[0]+'.rwa'
mesh_label = '{}'.format(tessellation_method) #, location_count, min_location_count)
#maps_label = 'sigma_{}_d_{}_v_{}_j_{:d}'.format(round(localization_error*1e3), diffusivity_smoothing, potential_smoothing, int(jeffreys_prior))

############################

positions_all=load_xyt(positions_file)
if trajectories_bool==True:
    positions_all=positions_all.drop(['n'],axis=1)
else:
    positions_all=positions_all.drop(['t'],axis=1)
    positions_all.columns=(['x','y','t'])
r_t=positions_all
#r_t=r_t.drop(['n'],axis=1)
#r_t.head()
r_t.tail()

NameError: name 'tessellation_method' is not defined

In [ ]:
# ts=np.sort(list(set(r_t['t'])))
# N_t=np.array([len(r_t[r_t['t']==t_].index) for t_ in ts])
# plt.plot(ts,N_t)
# plt.plot(ts[1:-1],N_t[1:-1])
# plt.plot([ts[0],ts[-1]],np.average(N_t[1:-1])*np.ones(2),'k--')
# plt.show()

# Deltas=np.diff(N_t[1:-1])
# plt.plot(ts[2:-1],Deltas)
# plt.show()
# print np.average(Deltas)

In [ ]:
# def mlnL_Deltas2(mus_,Deltas_):
#     mu_on=mus_[0]
#     mu_off=mus_[1]
#     return -sum([np.log(skellam.pmf(Delta,mu_on,mu_off)) for Delta in Deltas_])

# mu_on,mu_off=fmin(mlnL_Deltas2,x0=[100.,100.],args=(Deltas,))
# p_off=mu_off/np.average(N_t[1:-1])

# x_D=np.arange(np.floor(min(Deltas)),max(Deltas),1)
# x_D=np.arange(-50,51,1)
# h=5
# bins=np.arange(-50-h/2.,51,h)

# F=plt.figure()
# #F.set_size_inches(3.375*0.5,1.5)
# plt.axes([0.22,0.22,0.74,0.74])
# plt.hist(Deltas,label='Data',bins=bins)
# plt.plot(x_D,h*len(Deltas)*skellam.pmf(x_D,mu_on,mu_off),'r',label='Theory')
# plt.legend(loc=2,frameon=False)
# plt.xlabel(r'$\Delta$',labelpad=2)
# plt.ylabel(r'Count',labelpad=0)
# plt.axis([-50,50,0,100])
# plt.show()

mu_on=96.4809296249 
mu_off=96.4789899237
p_off=0.259937348758

print mu_on,mu_off,p_off

In [ ]:
t_start=0.04
t_end=1.04

if t_end==0.0:
    r_t=positions_all
else:
    r_t=positions_all[(positions_all['t']>=t_start)&(positions_all['t']<=t_end)]
#r_t

xlims=(34.,50.)
ylims=(24.,54.)
xlims=(37.,40.)
ylims=(37.,40.)
r_t=r_t[(r_t['x']>=xlims[0]) & (r_t['x']<=xlims[1]) & (r_t['y']>=ylims[0]) & (r_t['y']<=ylims[1])]

In [ ]:
ref_distance = 0.5 # [mu]

tessellation_method = 'hexagon'
tessellate(r_t, tessellation_method, ref_distance=ref_distance, min_location_count=0, output_file=rwa_file,\
           force=True, output_label=mesh_label)


In [ ]:
cell_plot(rwa_file, cell_indices=True, show=True)
#cell_plot(rwa_file, cell_indices=True, show=True, delaunay=True, voronoi=False)



In [ ]:
runtime=time()
#infer(rwa_file, 'nT_test02_optim', new_cell=Locations, dt=0.04, S=(2.*ref_distance)**2, p_off=p_off, mu_on=mu_on,\
#      times=times, method='NM', tol=1e-2)
infer(rwa_file, 'nT_test02_hex', new_cell=Locations, dt=0.04, S=(2.*ref_distance)**2, p_off=p_off, mu_on=mu_on,\
        method='NM', tol=1e-2)
print "\n Done!", time()-runtime, "s"

In [ ]:
map_plot(rwa_file,cmap='inferno') #, output_file='BP_1zone'+tessellation_method+','+str(2.*ref_distance)+'.png') 
#) #, label=(mesh_label, maps_label))